### Importing modules

In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageTk
import torch

### Detection Function for Image

In [2]:
def run_detect(filename):
    s = !python detect.py --source $filename --weights "C:\Users\nakul\Documents\Nakul\Machine Learning\Project\yolov5\best.pt" --conf 0.25
    item = []
    for i in range(len(s)):
        if('Done' in s[i]):
            item.append(s[i])

    for i in item:
        new = i.split()
        del new[:5]
        del new[-2:]
        for j in new:
            j = j[:-1]
    return j

### Image detection App

Run the below code to detect image 

In [4]:
window = tk.Tk()
window.title('Automated Checkout App')
window.geometry("640x480+10+10")
#bg = ImageTk.PhotoImage(Image.open("bg1.jpg").resize((1920,1080)))
#labelbg = tk.Label(image = bg)
#labelbg.place(x = 0, y = 0)
def open_image_file():
        filetypes = (
            ('JPG file', '*.jpg'),
            ('PNG file', '*.png'),
            ('All files', '*.*')
        )
        f = fd.askopenfile(filetypes=filetypes)
        filename = f.name
        image1 = Image.open(filename)
        image1 = image1.resize((300,300))
        test = ImageTk.PhotoImage(image1)
        label1 = tk.Label(image=test)
        label1.image = test
        label1.place(x=300, y=100)
        item = run_detect(filename)
        try:
            text_box.destroy()
        except:
            pass
        finally:
            text_box = tk.Label(window, text = 'Predicted Item: {}'.format(item), font = ("Montserrat", 12))
            text_box.place(x = 50, y = 150)
    
open_button = tk.Button(
            window,
            text='Open a File',
            command=open_image_file,
            font = ("Montserrat", 16)
        )
open_button.place(x = 250, y = 40)
window.mainloop()

### Video detection and Bill generation function

In [7]:
def run_detect_vdo(filename):
    s = !python detect.py --source $filename --weights "C:\Users\nakul\Documents\Nakul\Machine Learning\Project\yolov5\best.pt" --conf 0.25
    actual_list = []
    for i in range(len(s)):
      if('Done' in s[i]):
        actual_list.append(s[i])
    new_list = []
    for i in actual_list:
      new = i.split()
      del new[2]
      del new[-2:]
      items = new[4:]
      new_list.append(items)
    res = []
    [res.append(x) for x in new_list if x not in res]
    res = [ele for ele in res if ele != []]
    bill = {}
    classes = ['bread,','butter,','egg,','milk,', 'coconuts,']
    for i in range(len(res)):
      for j in res[i]:
        if(j in classes):
          index = res[i].index(j)
          if(j in list(bill.keys())):
            bill[j] = bill[j] + int(res[i][index-1])
          else:
            bill[j] = int(res[i][index-1])

    print(bill)
    import pandas as pd

    # per unit cost for each item 
    price = {'bread,' : 45 ,
             'butter,' : 60,
             'egg,': 50,
             'milk,': 30,
             'coconuts,': 15}

    # Creating the bill in a Dataframe
    df = pd.DataFrame()
    df['item(s)']=bill.keys()
    df['quantity']=bill.values()
    price_l=[]

    # Calculating the price of each item
    for i in range(len(df)):
      #print(df['item(s)'][i])
      p=price[df['item(s)'][i]]
      q=df['quantity'][i]
      p=p*q
      price_l.append(p)

    # Adding the price values to the dataframe 
    df['price']=price_l

    # Displaying the output in a formatted way
    print("Bill Generated"+'\n',df)
    print("\n\n"+"Grand total: ",sum(df['price']))

## Result

### Automated Checkout System with Bill Generation

Lets assume the video passed to the app is: 

![SegmentLocal](test.gif "segment")

In [10]:
window = tk.Tk()
window.title('Automated Checkout App')
window.geometry("640x480+10+10")
def open_image_file():
        filetypes = (
            ('MP4 file', '*.mp4'),
            ('All files', '*.*')
        )
        f = fd.askopenfile(filetypes=filetypes)
        filename = f.name
        run_detect_vdo(filename)
    
open_button = tk.Button(
            window,
            text='Open a File',
            command=open_image_file,
            font = ("Montserrat", 16)
        )
open_button.place(x = 250, y = 40)
window.mainloop()

{'bread,': 2, 'butter,': 1, 'coconuts,': 2, 'egg,': 1, 'milk,': 1}
Bill Generated
      item(s)  quantity  price
0     bread,         2     90
1    butter,         1     60
2  coconuts,         2     30
3       egg,         1     50
4      milk,         1     30


Grand total:  260


Here the bill generates as predicted and the model works